**欢迎关注Pyhton金融量化**  

此文件仅供金融量化知识星球圈友学习参考，请勿外传 

In [1]:
#引入可能用到的包
import pandas as pd  
import talib as ta
import numpy as np
from scipy import stats
import tushare as ts 
import matplotlib.pyplot as plt
%matplotlib inline   

#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

In [2]:
#获取股票代码
index={'上证综指': '000001.SH',
        '深证成指': '399001.SZ',
        '沪深300': '000300.SH',
        '创业板指': '399006.SZ',
        '上证50': '000016.SH',
        '中证500': '000905.SH',
        '中小板指': '399005.SZ',
        '上证180': '000010.SH'}

#获取当前交易的股票代码和名称
def get_code():
    df = pro.stock_basic(exchange='', list_status='L')
    codes=df.ts_code.values
    names=df.name.values
    stock=dict(zip(names,codes))
    #合并指数和个股成一个字典
    stocks=dict(stock,**index)
    return stocks    

In [3]:
# 获取交易数据
#使用之前先输入token，可以从个人主页上复制出来，每次调用数据需要先运行该命令
token='e0eeb08befd1f07516df2cbf9cbd58663f77fd72f92a04f290291c9d'
ts.set_token(token)
pro=ts.pro_api()
def get_daily_data(stock,start='',end=''):
    code=get_code()[stock]
    #如果代码在字典index里，则取的是指数数据
    if code in index.values():
        df=pro.index_daily(ts_code=code,start_date=start, end_date=end)
    #否则取的是个股数据
    else:
        df=pro.daily(ts_code=code, start_date=start, end_date=end)
    #将交易日期设置为索引值
    df.index=pd.to_datetime(df.trade_date)
    df=df.sort_index()
    df['date']=pd.to_datetime(df.trade_date)
    return df

In [23]:
# 交易策略，简单均线策略,输出每天的仓位
def strategy_ma(stock_data, short=5, long=20):
    """
    :param stock_data: 股票数据集
    :param short: 较短的窗口期
    :param long: 较长的窗口期
    :return: 当天收盘时持有该股票的仓位。

    最简单的均线策略。当天收盘后，短期均线上穿长期均线的时候，在第二天开盘买入。
     当短期均线下穿长期均线的时候，在第二天开盘卖出。每次都是全仓买卖.

    """
    # 计算短期和长期的移动平均线
    stock_data['ma_short'] = ta.MA(stock_data['close'], timeperiod=short)
    stock_data['ma_long'] = ta.MA(stock_data['close'], timeperiod=long)

    # 出现买入信号而且第二天开盘没有涨停
    stock_data.loc[(stock_data['ma_short'].shift(1) > stock_data['ma_long'].shift(1)) &
                  (stock_data['open'] < stock_data['close'].shift(1) * 1.097), 'position'] = 1
    # 出现卖出信号而且第二天开盘没有跌停
    stock_data.loc[(stock_data['ma_short'].shift(1) < stock_data['ma_long'].shift(1)) &
                  (stock_data['open'] > stock_data['close'].shift(1) * 0.903), 'position'] = 0

    stock_data['position'].fillna(method='ffill', inplace=True)
    stock_data['position'].fillna(0, inplace=True)
    stock_data['ret']=stock_data['pct_chg']/100
    return stock_data[['ts_code', 'date', 'open', 'close', 'ret', 'position']]

In [24]:
# 根据每日仓位计算总资产的日收益率
def position(df, slippage=1.0 / 1000, commision_rate=2.0 / 10000):
    """
    :param df: 股票账户数据集
    :param slippage: 买卖滑点 默认为1.0 / 1000
    :param commision_rate: 手续费 默认为2.0 / 10000
    :return: 返回账户资产的日收益率和日累计收益率的数据集
    """
    df.loc[df.index[0], 'capital_rtn'] = 0
    # 当加仓时,计算当天资金曲线涨幅capital_rtn.capital_rtn = 昨天的position在今天涨幅
    #+ #今天开盘新买入的position在今天的涨幅(扣除手续费)
    df.loc[df['position'] > df['position'].shift(1), 'capital_rtn'] = (df['close'] / df['open'] - 1) * (
        1 - slippage - commision_rate) * (df['position'] - df['position'].shift(1)) + df['ret'] * df[
        'position'].shift(1)
    # 当减仓时,计算当天资金曲线涨幅capital_rtn.capital_rtn = 今天开盘卖出的positipn
    #在今天的涨幅(扣除手续费) + #还剩的position在今天的涨幅
    df.loc[df['position'] < df['position'].shift(1), 'capital_rtn'] = (df['open'] / df['close'].shift(1) 
     - 1) * (1 - slippage - commision_rate) * (df['position'].shift(1) - df['position']) + df['ret'] * df['position']
    # 当仓位不变时,当天的capital_rtn是当天的change * position
    df.loc[df['position'] == df['position'].shift(1), 'capital_rtn'] = df['ret'] * df['position']
    return df

In [25]:
# 计算最近250天的股票,策略累计涨跌幅.以及每年（月，周）股票和策略收益
def period_return(stock_data, days=250, if_print=False):
    """
    :param stock_data: 包含日期、股票涨跌幅和总资产涨跌幅的数据集
    :param days: 最近250天
    :return: 输出最近250天的股票和策略累计涨跌幅以及每年的股票收益和策略收益
    """
    df = stock_data.loc[:,['ts_code','trade_date', 'ret', 'capital_rtn']]

    # 计算每一年(月,周)股票,资金曲线的收益
    year_rtn = df.loc[:,['ret', 'capital_rtn']].resample('A').apply(lambda x: (x + 1.0).prod() - 1.0)
    month_rtn = df.loc[:,['ret', 'capital_rtn']].resample('M').apply(lambda x: (x + 1.0).prod() - 1.0)
    week_rtn = df.loc[:,['ret', 'capital_rtn']].resample('W').apply(lambda x: (x + 1.0).prod() - 1.0)

    year_rtn.dropna(inplace=True)
    month_rtn.dropna(inplace=True)
    week_rtn.dropna(inplace=True)

    # 计算策略的年（月，周）胜率
    yearly_win_rate = len(year_rtn[year_rtn['capital_rtn'] > 0]) / len(year_rtn[year_rtn['capital_rtn'] != 0])
    monthly_win_rate = len(month_rtn[month_rtn['capital_rtn'] > 0]) / len(month_rtn[month_rtn['capital_rtn'] != 0])
    weekly_win_rate = len(week_rtn[week_rtn['capital_rtn'] > 0]) / len(week_rtn[week_rtn['capital_rtn'] != 0])

    # 计算股票的年（月，周）胜率
    yearly_win_rates = len(year_rtn[year_rtn['ret'] > 0]) / len(year_rtn[year_rtn['ret'] != 0])
    monthly_win_rates = len(month_rtn[month_rtn['ret'] > 0]) / len(month_rtn[month_rtn['ret'] != 0])
    weekly_win_rates = len(week_rtn[week_rtn['ret'] > 0]) / len(week_rtn[week_rtn['ret'] != 0])

    # 计算最近days的累计涨幅
    df = df.iloc[-days:]
    recent_rtn_line = df.loc[:,['trade_date']]
    recent_rtn_line['stock_rtn_line'] = (df['ret'] + 1).cumprod() - 1
    recent_rtn_line['strategy_rtn_line'] = (df['capital_rtn'] + 1).cumprod() - 1
    recent_rtn_line.reset_index(drop=True, inplace=True)

    # 输出
    if if_print:
        print ('\n最近' + str(days) + '天股票和策略的累计涨幅:')
        print (recent_rtn_line)
        print ('\n过去每一年股票和策略的收益:')
        print (year_rtn)
        print ('策略年胜率为：%f' % yearly_win_rate)
        print ('股票年胜率为：%f' % yearly_win_rates)
        print ('\n过去每一月股票和策略的收益:')
        print (month_rtn)
        print ('策略月胜率为：%f' % monthly_win_rate)
        print ('股票月胜率为：%f' % monthly_win_rates)
        print ('\n过去每一周股票和策略的收益:')
        print (week_rtn)
        print ('策略周胜率为：%f' % weekly_win_rate)
        print ('股票周胜率为：%f' % weekly_win_rates)
    return year_rtn, month_rtn, week_rtn, recent_rtn_line

In [26]:
# 根据每次买入的结果,计算相关指标
def trade_indicators(df):
    """
    :param df: 包含日期、仓位和总资产的数据集
    :return: 输出账户交易各项指标
    """
    # 计算资金曲线
    df['capital'] = (df['capital_rtn'] + 1).cumprod()
    # 记录买入或者加仓时的日期和初始资产
    df.loc[df['position'] > df['position'].shift(1), 'start_date'] = df['date']
    df.loc[df['position'] > df['position'].shift(1), 'start_capital'] = df['capital'].shift(1)
    df.loc[df['position'] > df['position'].shift(1), 'start_stock'] = df['close'].shift(1)
    # 记录卖出时的日期和当天的资产
    df.loc[df['position'] < df['position'].shift(1), 'end_date'] = df['date']
    df.loc[df['position'] < df['position'].shift(1), 'end_capital'] = df['capital']
    df.loc[df['position'] < df['position'].shift(1), 'end_stock'] = df['close']
    # 将买卖当天的信息合并成一个dataframe
    df_temp = df[df['start_date'].notnull() | df['end_date'].notnull()]

    df_temp['end_date'] = df_temp['end_date'].shift(-1)
    df_temp['end_capital'] = df_temp['end_capital'].shift(-1)
    df_temp['end_stock'] = df_temp['end_stock'].shift(-1)

    # 构建账户交易情况dataframe：'hold_time'持有天数，
    #'trade_return'该次交易盈亏,'stock_return'同期股票涨跌幅
    trade = df_temp.loc[df_temp['end_date'].notnull(), ['start_date', 'start_capital', 'start_stock',
                                                       'end_date', 'end_capital', 'end_stock']]
    trade.reset_index(drop=True, inplace=True)
    trade['hold_time'] = (trade['end_date'] - trade['start_date']).dt.days
    trade['trade_return'] = trade['end_capital'] / trade['start_capital'] - 1
    trade['stock_return'] = trade['end_stock'] / trade['start_stock'] - 1

    trade_num = len(trade)  # 计算交易次数
    max_holdtime = trade['hold_time'].max()  # 计算最长持有天数
    average_change = trade['trade_return'].mean()  # 计算每次平均涨幅
    max_gain = trade['trade_return'].max()  # 计算单笔最大盈利
    max_loss = trade['trade_return'].min()  # 计算单笔最大亏损
    total_years = (trade['end_date'].iloc[-1] - trade['start_date'].iloc[0]).days / 365
    trade_per_year = trade_num / total_years  # 计算年均买卖次数

    # 计算连续盈利亏损的次数
    trade.loc[trade['trade_return'] > 0, 'gain'] = 1
    trade.loc[trade['trade_return'] < 0, 'gain'] = 0
    trade['gain'].fillna(method='ffill', inplace=True)
    # 根据gain这一列计算连续盈利亏损的次数
    rtn_list = list(trade['gain'])
    successive_gain_list = []
    num = 1
    for i in range(len(rtn_list)):
        if i == 0:
            successive_gain_list.append(num)
        else:
            if (rtn_list[i] == rtn_list[i - 1] == 1) or (rtn_list[i] == rtn_list[i - 1] == 0):
                num += 1
            else:
                num = 1
            successive_gain_list.append(num)
    # 将计算结果赋给新的一列'successive_gain'
    trade['successive_gain'] = successive_gain_list
    # 分别在盈利和亏损的两个dataframe里按照'successive_gain'的值排序并取最大值
    max_successive_gain = trade[trade['gain'] == 1].sort_values(by='successive_gain', \
                        ascending=False)['successive_gain'].iloc[0]
    max_successive_loss = trade[trade['gain'] == 0].sort_values(by='successive_gain', \
                        ascending=False)['successive_gain'].iloc[0]

    #  输出账户交易各项指标
    print ('\n==============每笔交易收益率及同期股票涨跌幅===============')
    print (trade[['start_date', 'end_date', 'trade_return', 'stock_return']])
    print ('\n====================账户交易的各项指标=====================')
    print ('交易次数为：%d   最长持有天数为：%d' % (trade_num, max_holdtime))
    print ('每次平均涨幅为：%f' % average_change)
    print ('单次最大盈利为：%f  单次最大亏损为：%f' % (max_gain, max_loss))
    print ('年均买卖次数为：%f' % trade_per_year)
    print ('最大连续盈利次数为：%d  最大连续亏损次数为：%d' % (max_successive_gain, max_successive_loss))
    return trade

In [27]:
# 计算年化收益率函数
def annual_return(date_line, capital_line):
    """
    :param date_line: 日期序列
    :param capital_line: 账户价值序列
    :return: 输出在回测期间的年化收益率
    """
    # 将数据序列合并成dataframe并按日期排序
    df = pd.DataFrame({'date': date_line, 'capital': capital_line})
    # 计算年化收益率
    annual = (df['capital'].iloc[-1] / df['capital'].iloc[0]) ** (250 / len(df)) - 1
    return annual

# 计算最大回撤函数
def max_drawdown(date_line, capital_line):
    df = pd.DataFrame({'date': date_line, 'capital': capital_line})
    #计算最大回撤
    max_dd = ((df['capital'].cummax()-df['capital']) / df['capital'].cummax()).max()  
    return max_dd

In [30]:
pd.options.mode.chained_assignment = None
# =====读取数据
df0=get_daily_data('沪深300',start='20150101')
# =====根据策略,计算仓位,资金曲线等
# 计算买卖信号
stock_data=strategy_ma(df0, short=5, long=20)
# 计算策略每天涨幅
return_data = position(stock_data)
# 计算资金曲线
return_data['capital'] = (return_data['capital_rtn'] + 1).cumprod()
# =====根据策略结果,计算评价指标
# 计算最近250天的股票,策略累计涨跌幅.以及每年（月，周）股票和策略收益
period_return(return_data, days=250, if_print=True)
# 根据每次买卖的结果,计算相关指标
trade_indicators(stock_data)
# =====根据资金曲线,计算相关评价指标
date_line = list(return_data['date'])
capital_line = list(return_data['capital'])
stock_line = list(return_data['close'])
print (f'\n股票的年化收益为：{annual_return(date_line, stock_line)}')
print (f'策略的年化收益为：{annual_return(date_line, capital_line)}')
print (f'\n股票最大回撤: {max_drawdown(date_line, stock_line)}')
print (f'策略最大回撤: {max_drawdown(date_line, capital_line)}')


最近250天股票和策略的累计涨幅:
     trade_date  stock_rtn_line  strategy_rtn_line
0           NaN       -0.011601          -0.011601
1           NaN       -0.015799          -0.015799
2           NaN       -0.017517          -0.017517
3           NaN       -0.016797          -0.016797
4           NaN       -0.036437          -0.036437
5           NaN       -0.057800          -0.057800
6           NaN       -0.073361          -0.058998
7           NaN       -0.085103          -0.058998
8           NaN       -0.058384          -0.058998
9           NaN       -0.073579          -0.058998
10          NaN       -0.050374          -0.058998
11          NaN       -0.048281          -0.058998
12          NaN       -0.052383          -0.058998
13          NaN       -0.057254          -0.058998
14          NaN       -0.079876          -0.058998
15          NaN       -0.084138          -0.058998
16          NaN       -0.097305          -0.058998
17          NaN       -0.086787          -0.058998
18          